# Análisis automático orientado a corrección de textos

En este cuaderno estoy probando diferentes **dimensiones de corrección y análisis de textos** mediante el uso de modelos de lenguaje (Azure OpenAI).  

El objetivo es evaluar y detectar errores en tres grandes dimensiones:

1. **Ortografía**  
   - Acentual: reglas ortográficas que rigen la acentuación de la lengua española.
   - Literal: omisiones, sustituciones, adiciones de letras, mayúsculas.  
   - Puntual: uso incorrecto de comas, puntos, signos de interrogación y exclamación, etc. 

2. **Vocabulario**  
   - Amplitud: detección de repeticiones innecesarias dentro de un mismo párrafo, considerando sustantivos, adjetivos, verbos y adverbios.  
   - Precisión léxica: reconocimiento del uso incorrecto del significado de palabras, de los extranjerismos incorrectamento formulados y de palabras e ideas reiteradadas excesivamente.
   - Adecuación al registro de habla formal: identificación del empleo de modo de comunicación escrita que se ajusta a la norma culta de la lengua, caracterizado por un uso cuidadoso y preciso del vocabulario y la gramática.

3. **Cohesión textual**  
   - Concordancia gramatical: sujeto-predicado, sustantivo-adjetivo, tiempos verbales en enumeraciones y cláusulas condicionales.  
   - Conexión textual: uso de preposiciones, locuciones, conectores adecuados y organizadores textuales.  
   - Referencias y correferencias: detección de pronombres o expresiones sin referente claro, como *esto*, *lo anterior*, etc.  

Cada dimensión se prueba sobre los textos del dataset **`escritos_seleccionados_a1.xlsx`**, y los resultados se almacenan en archivos JSON, manteniendo un registro estructurado de los errores detectados.  

El flujo de trabajo consiste en evaluar cada subdimensión mediante llamadas a la API de Azure OpenAI, siguiendo los criterios definidos en la rúbrica elaborada por expertos (versión del 15 de septiembre de 2025), la cual establece tanto los aspectos de evaluación como los puntajes asociados. Para cada una de las subdimensiones se realiza una llamada independiente por cada uno de los 10 textos contenidos en el documento escritos_seleccionados_ort.xlsx.

*Documento elaborado por Renata Ramírez, practicante MIDEUC.*

## 1. Ortografía

### Acentual

In [1]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_acentual_template = """
Identifica EXCLUSIVAMENTE los errores relacionados con el uso de tildes en español en un texto que te entregaré, retornando un diccionario JSON válido según la estructura que se detallará más adelante.

Considera las reglas generales (palabras agudas, graves, esdrújulas y sobreesdrújulas) y las reglas especiales (acento diacrítico y acento dierético) que rigen el uso de tildes en español.

Tu respuesta debe ser un JSON válido que siga exactamente esta estructura:

{{
  "errores_acentuales": {{
        "error_1": ["error → corrección"],
        "error_2": ["error → corrección"],
        "error_3": ["error → corrección"],
        ...
  }}
}}

### Texto a evaluar:
{essay}

### INSTRUCCIONES CRÍTICAS:
- SOLO reporta errores en los que la diferencia entre error y corrección sea la presencia o ausencia de una tilde.
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON.
"""

resultados = []

for i, texto in enumerate(textos, start=1):    
    prompt_acentual = prompt_acentual_template.format(essay=texto)

    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": 'Realiza las tareas de corrección que te solicite para un texto entregado'},
        {"role": "user", "content": prompt_acentual}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0, (No es soportado en Azure OpenAI)
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "02")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "ort_acentual.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\02\ort_acentual.json


### Literal

In [2]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_literal_template = """
Identifica los errores EXCLUSIVAMENTE según las instrucciones que se detallarán, para un texto que te entregaré. Debes retornar un diccionario JSON válido según la estructura que se detallará más adelante.

Corrige ÚNICAMENTE los siguientes tipos de errores:

1. Mayúsculas ausentes: inicio de párrafo, tras punto, nombres propios. Ej: ". ahora" → ". Ahora", "madrid" → "Madrid".
2. Omisión de letras: ej. "alchol" → "alcohol", "gerra" → "guerra", "aora" → "ahora".
3. Sustitución de letras: ej. "girafa" → "jirafa", "obeja" → "oveja".
4. Adición de letras: ej. "rrallado" → "rallado", "exhuberante" → "exuberante". Incluye "en carro": ej. "apartir" → "a partir", "através" → "a través".

Tu respuesta debe ser un JSON válido que siga exactamente esta estructura:

{{
  "errores_ortografia_literal": {{
        "error_1": ["error → corrección"],
        "error_2": ["error → corrección"],
        ...
  }}
}}

### Texto a evaluar
{essay}

### INSTRUCCIONES CRÍTICAS:
- NO revises ni corrijas tildes.
- NO incluyas el texto completo corregido.
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON.
"""

resultados = []

for i, texto in enumerate(textos, start=1):
    prompt_literal = prompt_literal_template.format(essay=texto)

    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": 'Realiza las tareas de corrección que te solicite para un texto entregado'},
        {"role": "user", "content": prompt_literal}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0,
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "02")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "ort_literal.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\02\ort_literal.json


### Puntual

In [3]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_puntual_template = """
Identifica los errores EXCLUSIVAMENTE según las instrucciones que se detallarán, para un texto que te entregaré. Debes retornar un diccionario JSON válido según la estructura que se detallará más adelante.

Tu tarea es corregir ÚNICAMENTE errores relacionados con signos de puntuación, de acuerdo con las normas de la RAE:

- Incorrecta presencia de una coma u omisión de una coma requerida.
- Incorrecta presencia de un punto u omisión de un punto requerido (final, seguido o aparte).
- Incorrecta presencia de un punto y coma. NO sanciones cuando la ausencia de punto y coma.
- Incorrecta presencia u omisión de signos de exclamación e interrogación.

Tu respuesta debe ser un JSON válido que siga exactamente esta estructura:

{{
  "errores_ortografia_puntual": {{
        "error_1": ["error → corrección"],
        "error_2": ["error → corrección"],
        ...
  }}
}}

### Instrucciones de la estructura:
- Ante un error de puntuación, el 'error' y 'corrección' en el JSON deben contener la palabra que antecede al signo faltante y la que lo sigue. Por ejemplo, si el texto decía "Para lograr un desarrollo sostenible es necesario cambiar nuestro hábitos de consumo", "error_i" debe ser ["sostenible es → sostenible, es"]

### Texto a evaluar:
{essay}

### Instrucciones CRÍTICAS:
- NO revises ni corrijas tildes
- Cada error y corrección debe mostrarse con el fragmento MÁS BREVE posible que contenga el error (no más de 20 caracteres).
- NO incluyas el texto completo corregido.
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON.
"""


resultados = []

for i, texto in enumerate(textos, start=1):
    prompt_puntual = prompt_puntual_template.format(essay=texto)

    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": 'Realiza las tareas de corrección que te solicite para un texto entregado'},
        {"role": "user", "content": prompt_puntual}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0,
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "02")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "ort_puntual.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\02\ort_puntual.json


## 2. Vocabulario

### Amplitud

In [4]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_amplitud_template = """
Cuenta las palabras repetidas siguiendo las instrucciones que se detallarán, para un texto que te entregaré. Debes retornar un diccionario JSON válido según la estructura que se detallará más adelante.

Deberas contar las palabras repetidas en el texto, siguiendo las siguientes reglas y excepciones.

- Solo contabilizar repeticiones en SUSTANTIVOS, ADJETIVOS, VERBOS y ADVERBIOS. No incluyas conectores.
- NO cuentes palabras repetidas usadas como figura retórica de énfasis, lo que se advierte rápidamente por el sentido del texto.  
- No contar conjugaciones verbales ni perífrasis verbales (auxiliar + infinitivo/gerundio/participio).

Tu respuesta debe ser un JSON válido que siga exactamente esta estructura:

{{
  "errores_vocabulario_amplitud": {{
        "palabra1": ["n veces"],
        "palabra2": ["n veces"]
  }}
}}

### Texto a evaluar
{essay}

### Instrucciones CRÍTICAS:
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON.
"""


resultados = []

for i, texto in enumerate(textos, start=1):
    print(f"Procesando texto {i}/{len(textos)}...")

    parrafos = [p.strip() for p in texto.split("\n") if p.strip()]  

    errores_por_texto = {"id_texto": i, "errores_vocabulario_amplitud": {}}

    for j, parrafo in enumerate(parrafos, start=1):
        prompt_amplitud = prompt_amplitud_template.format(essay=parrafo)
        
        messages = [
            {"role": "system", "content": 'Realiza las tareas de corrección que te solicite para un texto entregado'},
            {"role": "user", "content": prompt_amplitud}
        ]

        try:
            response = client.chat.completions.create(
                model=DEPLOYMENT_GPT,
                messages=messages,
                # temperature=0,
                response_format={"type": "json_object"}
            )

            respuesta_limpia = response.choices[0].message.content.strip()
            resultado_json = json.loads(respuesta_limpia)

            errores_por_texto["errores_vocabulario_amplitud"][f"parrafo_{j}"] = resultado_json.get("errores_vocabulario_amplitud", {})

        except json.JSONDecodeError as e:
            print(f"Error al parsear JSON en texto {i}, párrafo {j}: {e}")
            print("Respuesta cruda recibida:")
            print(respuesta_limpia)

    resultados.append(errores_por_texto)

output_dir = os.path.join("..", "resultados", "02")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "voc_amplitud.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\02\voc_amplitud.json


### Precisión léxica

In [5]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_precision_template = """
Identifica los errores EXCLUSIVAMENTE según las instrucciones que se detallarán, para un texto que te entregaré. Debes retornar un diccionario JSON válido según la estructura que se detallará más adelante.

Identifica como error si se cumple alguno de los siguientes criterios:

A) USO INCORRECTO DE SIGNIFICADOS:
- Detecta palabras usadas con un significado equivocado que cambien el sentido del texto. La palabra usada debe existir y estar correctamente escrita.
- Ejemplo: “absceso” por “acceso”, “actitud” por “aptitud”.

B) EXTRANJERISMOS:
- Señala extranjerismos que NO estén en comillas, cursiva o introducidos explícitamente por el autor.
- Ejemplo incorrecto: usar "online" sin comillas ni contexto en vez de “en línea”.
- No sanciones extranjerismos correctos o presentados como citas.

C) REDUNDANCIAS:
- Detecta repeticiones innecesarias de palabras o ideas.
- Ejemplo: “salir para afuera”, “volar por los aires”, “hace un tiempo atrás”.

Tu respuesta debe ser un JSON válido que siga exactamente esta estructura:

{{
  "errores_precision": {{
        "error_1": ["error → corrección", CRITERIO],
        "error_2": ["error → corrección", CRITERIO],
        ...,
        "error_n": ["error → corrección", CRITERIO]
  }}
}}

### Reglas para la estructura
- Muestra el fragmento MÁS BREVE posible con el error
- En CRITERIO indica "USO INCORRECTO DE SIGNIFICADOS", "EXTRANJERISMO" o "REDUNDANCIA".

### Texto a evaluar
{essay}

REGLAS:
- Enumera todos los errores encontrados bajo la clave "errores_vocabulario_precision".
- NO incluyas el texto completo corregido.
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON.
"""


resultados = []

for i, texto in enumerate(textos, start=1):
    prompt_precision = prompt_precision_template.format(essay=texto)
    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": 'Realiza las tareas de corrección que te solicite para un texto entregado'},
        {"role": "user", "content": prompt_precision}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0,
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "02")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "voc_precision.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\02\voc_precision.json


### Registro formal

In [6]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_formal_template = """
Identifica los errores EXCLUSIVAMENTE según las instrucciones que se detallarán, para un texto que te entregaré. Debes retornar un diccionario JSON válido según la estructura que se detallará más adelante.

Sanciona como error únicamente los siguientes casos:

1) SEGUNDA PERSONA SINGULAR:
Sanciona el uso del pronombre “tú” y/o formas verbales conjugadas en segunda persona singular PARA EXPONER GENERALIZACIONES o PROCEDIMIENTOS QUE DEBERÍAN EXPRESARSE EN FORMA IMPERSONAL o GENERAL.
- Por ejemplo: "Cuando postulas a un trabajo, tú tienes que pasar una entrevista".

2) ABREVIACIONES:
Sanciona palabras abreviadas propias de la oralidad.
- Por ejemplo: “profe” por “profesor/a”, “finde” por “fin de semana”.

3) COLOQUIALISMOS:
Sanciona el uso de expresiones coloquiales (expresiones propias de la conversación informal)
- Por ejemplo: “socio” por “amigo”, “cachar” por “comprender” o “darse cuenta”, “pega” por “trabajo”; el uso de calificativos más expresivos: “mina” por “guapa”, “bacán” por “excelente”, “pichiruche” por “insignificante”; el uso de diminutivos y el prefijo “super” para amplificar valor.

4) CÓDIGO ESCRITO NO FORMAL:
Sanciona la sustitución de letras por símbolos para representar palabras
- Por ejemplo: "q’" por "que", "total%" por "totalmente", "publicaØ" por "publicación").

Tu respuesta debe ser un JSON válido que siga exactamente esta estructura::

{{
  "errores_vocabulario_formal": {{
        "error_1": ["error → corrección sugerida", CRITERIO],
        "error_2": ["error → corrección sugerida", CRITERIO],
        ...,
        "error_n": ["error → corrección sugerida", CRITERIO]
  }}
}}

### Reglas sobre el formato:
- En "CRITERIO", indica si el criterio es "SEGUNDA PERSONA SINGULAR", "ABREVIACIONES", "COLOQUIALISMOS" o "CÓDIGO ESCRITO NO FORMAL".
- En cada caso, muestra el fragmento MÁS BREVE posible con el error.
- Entrega una corrección sugerida acorde al registro formal.

### Texto a analizar:
{essay}

### Importante:
- NO incluyas el texto completo corregido.
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON.
"""

resultados = []

for i, texto in enumerate(textos, start=1):
    prompt_formal = prompt_formal_template.format(essay=texto)

    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": 'Realiza las tareas de corrección que te solicite para un texto entregado'},
        {"role": "user", "content": prompt_formal}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            #temperature=0,
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "02")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "voc_formal.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

# 4) CÓDIGO ESCRITO NO FORMAL:
# - Detecta usos innecesarios de caracteres o símbolos en el español, sin incluiir til
# que alteren la escritura formal con símbolos o caracteres que no son letras ni tildes.
# - Ejemplos: "q’" por "que", "total%" por "totalmente", "publicaØ" por "publicación".

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\02\voc_formal.json


## 3. Cohesión

### Concordancia gramatical

In [7]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_concordancia_template = """
Identifica los errores EXCLUSIVAMENTE según las instrucciones que se detallarán, para un texto que te entregaré. Debes retornar un diccionario JSON válido según la estructura que se detallará más adelante.

Consideraremos faltas de concordancia gramatical SOLO los siguientes casos:

1) Falta de concordancia (en género o número) entre **sustantivo y adjetivo**.

2) Falta de concordancia (en género o número) entre **sujeto y predicado**.

3) Falta de concordancia (en género o número) entre **pronombres relativos y sus antecedentes**.

Faltas de consistencia verbal SOLO en los siguientes casos:

A) Cambio de tiempo verbal en una enumeración de acciones. Por ejemplo "Se MIDE el nivel de agua, se OBSERVAN los cambios en la sustancia y se REGISTRARÁ el proceso por escrito".

B) Uso de tiempos no adecuados en cláusulas condicionales y concesivas. Por ejemplo: "Si me HUBIERAN invitado, HABRÍA venido" o "Aunque los químicos se HUBIERAN resguardado de la temperatura, igualmente HABRÍAN perdido su calidad".

Tu respuesta debe ser un JSON válido que siga exactamente esta estructura:  

{{
  "errores_cohesion_concordancia": {{
    "error_1": ["error → corrección", "tipo: sustantivo-adjetivo"],
    "error_2": ["error → corrección", "tipo: sujeto-predicado"],
    "error_3": ["error → corrección", "tipo: pronombre-relativo"],
    "error_4": ["error → corrección", "tipo: A"],
    "error_5": ["error → corrección", "tipo: B"],
    ...
    "error_n": ["error → corrección", "tipo: ..."]
  }}
}}

### Reglas sobre la estructura:
- En tipo de error indica "sustantivo-adjetivo", "sujeto-predicado", "pronombre-relativo", "A" (enumeración de acciones) o "B" (cláusulas).
- En cada caso, muestra el fragmento MÁS BREVE posible con el error.
- Entrega la corrección del error.

### Texto a analizar:
{essay}

### Instrucciones críticas:  
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON.  
"""

resultados = []

for i, texto in enumerate(textos, start=1):
    prompt_concordancia = prompt_concordancia_template.format(essay=texto)

    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": 'Realiza las tareas de corrección que te solicite para un texto entregado'},
        {"role": "user", "content": prompt_concordancia}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            #temperature=0,
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "02")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "coh_concordancia.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\02\coh_concordancia.json


### Conexión textual

In [8]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_conexion_template = """
Identifica los errores EXCLUSIVAMENTE según las instrucciones que se detallarán, para un texto que te entregaré. Debes retornar un diccionario JSON válido según la estructura que se detallará más adelante.

Se consideran errores únicamente los siguientes casos:

1. Uso inadecuado de preposiciones o rección: falta la preposición o uso de una preposición incorrecta.  
   - Incluye queísmo (uso de "que" sin "de" cuando corresponde) y dequeísmo (uso de "de que" cuando solo corresponde "que").  
2. Uso inadecuado de locuciones preposicionales u otras construcciones lexicales de carácter conectivo (ej.: "sobre la base de", "a razón de", "en relación con") **cuando afecten la lógica de conexión oracional y/o textual**.  
3. Ausencia de conectores o uso de conectores inadecuados para expresar la idea que se busca; mal uso de organizadores textuales (ej.: "en primer lugar", "por un lado", etc.).

Tu respuesta debe ser un JSON válido que siga exactamente esta estructura: 

{{
  "errores_cohesion_conexion": {{
    "error_1": ["error → corrección", "tipo: preposición/rección"],
    "error_2": ["error → corrección", "tipo: queísmo"],
    "error_3": ["error → corrección", "tipo: dequeísmo"],
    "error_4": ["error → corrección", "tipo: locución-preposicional"],
    "error_5": ["error → corrección", "tipo: conector-ausente"],
    "error_6": ["error → corrección", "tipo: conector-inadecuado"],
    ...
    "error_n": ["error → corrección", "tipo: ..."]
  }}
}}

### Reglas sobre la estructura:
- En tipo de error indica "preposición/rección", "queísmo", "dequeísmo", "locución-preposicional", "conector-ausente" o "conector-inadecuado".
- En cada caso, muestra el fragmento MÁS BREVE posible con el error.

### Texto a evaluar:
{essay}

### Importante:
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON.
"""

resultados = []

for i, texto in enumerate(textos, start=1):
    prompt_conexion = prompt_conexion_template.format(essay=texto)
    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": 'Realiza las tareas de corrección que te solicite para un texto entregado'},
        {"role": "user", "content": prompt_conexion}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0,
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "02")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "coh_conexion.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\02\coh_conexion.json


### Referencias y correferencias

In [9]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_referencias_template = """  
Identifica los errores EXCLUSIVAMENTE según las instrucciones que se detallarán, para un texto que te entregaré. Debes retornar un diccionario JSON válido según la estructura que se detallará más adelante.  

Considera errores únicamente según el siguiente criterio:

- Pronombres o mecanismos de correferencia que no tengan un referente claro en el texto. Por ejemplo: "esto", "lo anterior", "esa situación".  

Tu respuesta debe ser un JSON válido que siga exactamente esta estructura:

{{
  "errores_cohesion_referencia": {{
    "error_1": ["error → corrección", "tipo: pronombre-sin-referencia"],
    "error_2": ["error → corrección", "tipo: pronombre-sin-referencia"],
    ...
    "error_n": ["error → corrección", "tipo: pronombre-sin-referencia"]
  }}
}}

### Reglas de la estructura:
- Cada error debe mostrarse con el fragmento MÁS BREVE posible que contenga el error (no frases completas).
- En 'tipo' entrega pronombre-sin-referencia para todos los errores

### Texto a evaluar:
{essay}

### Instrucciones criticas 
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON. 
"""

resultados = []

for i, texto in enumerate(textos, start=1):
    prompt_referencias = prompt_referencias_template.format(essay=texto)

    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": 'Realiza las tareas de corrección que te solicite para un texto entregado'},
        {"role": "user", "content": prompt_referencias}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0,
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "02")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "coh_referencias.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\02\coh_referencias.json
